In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.ndimage import convolve
from math import pi, sin, cos, tan

In [2]:
dir_path = "C:/Users/gerag/Desktop/Coursera/Fisrt Principles of Computer Vision/Features and Boundaries/Edge Detection/"
im_name = "rubiks.png"

In [3]:
img_color = cv2.imread(dir_path+im_name) #Read image in grey scale
img = cv2.cvtColor(img_color,cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(img,127,255,0)
Moments = cv2.moments(thresh)

In [5]:
Moments

{'m00': 236379645.0,
 'm10': 142125368175.0,
 'm01': 141187784940.0,
 'm20': 119674091862075.0,
 'm11': 85559121273390.0,
 'm02': 120390502976070.0,
 'm30': 1.1312702240332491e+17,
 'm21': 7.249772799886307e+16,
 'm12': 7.317957331325094e+16,
 'm03': 1.1503797517626922e+17,
 'mu20': 34219947623558.598,
 'mu11': 668707310672.4406,
 'mu02': 36060100431199.06,
 'mu30': 21811179401028.445,
 'mu21': 213160136965666.7,
 'mu12': -5117222101350.206,
 'mu03': 52779000880812.57,
 'nu20': 0.0006124338522863045,
 'nu11': 1.1967843984810537e-05,
 'nu02': 0.0006453670374909144,
 'nu30': 2.5389488974228146e-08,
 'nu21': 2.4813087122558655e-07,
 'nu12': -5.956745929785963e-09,
 'nu03': 6.143784507457597e-08}